In [1]:
from pprint import pprint

import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

from triagerx.model.lbt_p_deberta import LBTPDeberta
from triagerx.loss.loss_functions import *
from triagerx.system.triagerx import TriagerX

/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Load Data

In [2]:
train_data_path = "/home/mdafifal.mamun/notebooks/triagerX/train_data.csv"
test_data_path = "/home/mdafifal.mamun/notebooks/triagerX/test_data.csv"

train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

In [3]:
train_df["raw_text"] = train_df.apply(
            lambda x: "Title: "
            + str(x["issue_title"])
            + "\nDescription: "
            + str(x["description"]),
            axis=1,
        )

test_df["raw_text"] = test_df.apply(
            lambda x: "Title: "
            + str(x["issue_title"])
            + "\nDescription: "
            + str(x["description"]),
            axis=1,
        )

### Developer Mapping

In [4]:
vm_users = ['pshipton', 'keithc-ca', 'gacholio', 'tajila', 'babsingh', 'JasonFengJ9', 'fengxue-IS', 'hangshao0', 'theresa.t.mammarella', 'ChengJin01', 'singh264', 'thallium', 'ThanHenderson']
jvmti_users = ['gacholio', 'tajila', 'babsingh', 'fengxue-IS']
jclextensions_users = ['JasonFengJ9', 'pshipton', 'keithc-ca']
test_users = ['LongyuZhang', 'annaibm', 'sophiaxu0424', 'KapilPowar', 'llxia']
build_users = ['adambrousseau', 'mahdipub']
gc_users = ['dmitripivkine', 'amicic', 'kangyining', 'LinHu2016']

# Putting them in dictionaries
components_developers_map = {
    'comp:vm': vm_users,
    'comp:jvmti': jvmti_users,
    'comp:jclextensions': jclextensions_users,
    'comp:test': test_users,
    'comp:build': build_users,
    'comp:gc': gc_users
}

expected_users = [user for user_list in components_developers_map.values() for user in user_list]

In [5]:
train_df = train_df[train_df["owner"].isin(expected_users)]
print("Total train issues after developer filtering:", len(train_df))

test_df = test_df[test_df["owner"].isin(expected_users)]
print("Total train issues after developer filtering:", len(test_df))

Total train issues after developer filtering: 1577
Total train issues after developer filtering: 398


In [6]:
developer_id_map = dict(zip(train_df["owner"], train_df["owner_id"]))
print(developer_id_map)

{'pshipton': 12, 'JasonFengJ9': 1, 'llxia': 11, 'keithc-ca': 10, 'hangshao0': 9, 'fengxue-IS': 7, 'babsingh': 5, 'LinHu2016': 2, 'thallium': 15, 'tajila': 14, 'amicic': 4, 'LongyuZhang': 3, 'gacholio': 8, 'dmitripivkine': 6, 'ChengJin01': 0, 'singh264': 13}


In [7]:
component_id_map = {'comp:build': 0, 'comp:gc': 1, 'comp:jclextensions': 2, 'comp:jvmti': 3, 'comp:test': 4, 'comp:vm': 5}

print(component_id_map)

{'comp:build': 0, 'comp:gc': 1, 'comp:jclextensions': 2, 'comp:jvmti': 3, 'comp:test': 4, 'comp:vm': 5}


In [8]:
train_df = train_df[train_df["component"].isin(component_id_map.keys())]
test_df = test_df[test_df["component"].isin(component_id_map.keys())]

In [9]:
train_df["component_id"] = train_df["component"].apply(lambda x: component_id_map[x])
test_df["component_id"] = test_df["component"].apply(lambda x: component_id_map[x])

# Load Models

In [10]:
len(test_df.component_id.unique())

6

### Prepare Tokenizer

In [11]:
import yaml

In [12]:
with open("/home/mdafifal.mamun/notebooks/triagerX/triagerx/config/config.yaml", "r") as file:
    configs = yaml.safe_load(file)

In [13]:
tokenizer_config = configs["tokenizer"]

In [14]:
special_tokens = {
    "hex": "[HEX]",
    "timestamp": "[TIMESTAMP]",
    "float": "[FLOAT_VALUE]",
    "param": "[PARAM_VALUE]"
}

tokenizer = AutoTokenizer.from_pretrained(tokenizer_config["model_name"])
special_tokens_dict = {"additional_special_tokens": list(special_tokens.values())}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [15]:
def get_trained_model(weight_path: str, num_classes, unfrozen_layers, dropout, base_model, tokenizer):
    model = LBTPDeberta(
        output_size=num_classes, 
        unfrozen_layers=unfrozen_layers, 
        dropout=dropout, 
        base_model=base_model
    )

    model.base_model.resize_token_embeddings(len(tokenizer))

    model.load_state_dict(torch.load(weight_path))

    return model

In [16]:
component_model_weights = "/work/disa_lab/projects/triagerx/models/testing/comp_ex_LBTPDeberta_u5_6_classes_CombinedLoss.pt"
developer_model_weights = "/work/disa_lab/projects/triagerx/models/testing/dev_ex_LBTPDeberta_u4_cv9_classes16_CombinedLoss.pt"

In [17]:
len(test_df.component_id.unique())

6

In [18]:
component_model = get_trained_model(
    weight_path=component_model_weights,
    num_classes=len(test_df.component_id.unique()), 
    unfrozen_layers=5, 
    dropout=0.2, 
    base_model="microsoft/deberta-base",
    tokenizer=tokenizer
)


/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/mdafifal.mamun/miniconda3/envs/trx/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [19]:
developer_model = get_trained_model(
    weight_path=developer_model_weights,
    num_classes=len(train_df.owner_id.unique()), 
    unfrozen_layers=4, 
    dropout=0.3, 
    base_model="microsoft/deberta-large",
    tokenizer=tokenizer
)

In [20]:
similarity_model = SentenceTransformer('all-mpnet-base-v2')

In [21]:
trx = TriagerX(
    developer_prediction_model=developer_model,
    component_prediction_model=component_model,
    similarity_model=similarity_model,
    tokenizer=tokenizer,
    tokenizer_config=tokenizer_config,
    train_data=train_df,
    developer_id_map=developer_id_map,
    component_id_map=component_id_map,
    component_developers_map=components_developers_map,
    issues_path="/home/mdafifal.mamun/notebooks/triagerX/notebook/data/openj9/issue_data",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

2024-05-02 23:29:28.117 | DEBUG    | triagerx.system.triagerx:__init__:53 - Using device: cuda
2024-05-02 23:29:28.119 | DEBUG    | triagerx.system.triagerx:__init__:54 - Generating embedding for existing issues...


In [22]:
def get_recommendation_for_test_index(test_df, test_idx):
    actual_owner = test_df.iloc[test_idx].owner
    actual_component = test_df.iloc[test_idx].component
    issue_url = test_df.iloc[test_idx].issue_url

    print("Issue:", test_df.iloc[test_idx].issue_url)
    print("Actual component:", actual_component)
    print("Actual owner:", actual_owner)
    recommendation = trx.get_recommendation(
        test_df.iloc[test_idx].text,
        k_comp=3,
        k_dev=3,
        k_rank=5,
        similarity_threshold=0.5
    )

    status = None

    if actual_owner in recommendation["predicted_developers"]:
        status = "Matched"
    else:
        status = "Not Matched"

    recommendation["actual_owner"] = actual_owner
    recommendation["issue_url"] = issue_url
    recommendation["actual_component"] = actual_component
    recommendation["model_pred_status"] = status
    recommendation["rank_status"] = "Matched" if actual_owner in recommendation["combined_ranking"][:3] else "Not Matched"

    return recommendation

In [23]:
recommendations_matched = []
component_matched = []
ranked_matched = []

for idx in range(len(test_df)):
    rec = get_recommendation_for_test_index(test_df, idx)
    
    if rec["model_pred_status"] == "Matched":
        recommendations_matched.append(rec)
    
    if rec["actual_component"] in rec["predicted_components"]:
        component_matched.append(rec)

    if rec["rank_status"] == "Matched":
        ranked_matched.append(rec)

# rec_df = pd.DataFrame(recommendations_matched)
# rec_df.to_csv("recommendations_matched.csv")

# rec_df = pd.DataFrame(recommendations_unmatched)
# rec_df.to_csv("recommendations_unmatched.csv")

2024-05-02 23:29:36.457 | DEBUG    | triagerx.system.triagerx:get_recommendation:58 - Processing issue...
2024-05-02 23:29:36.462 | DEBUG    | triagerx.system.triagerx:get_recommendation:62 - Prediciting components...


Issue: https://github.com/eclipse-openj9/openj9/issues/5796
Actual component: comp:vm
Actual owner: pshipton


2024-05-02 23:29:36.674 | INFO     | triagerx.system.triagerx:get_recommendation:65 - Predicted components: ['comp:vm', 'comp:test', 'comp:jvmti']
2024-05-02 23:29:36.676 | DEBUG    | triagerx.system.triagerx:get_recommendation:67 - Generating developer recommendation...
2024-05-02 23:29:36.735 | INFO     | triagerx.system.triagerx:get_recommendation:70 - Recommended developers: ['pshipton', 'JasonFengJ9', 'hangshao0', 'keithc-ca', 'tajila']
2024-05-02 23:29:36.737 | DEBUG    | triagerx.system.triagerx:get_recommendation:72 - Generating recommendation by similarity...
2024-05-02 23:29:36.762 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:181 - Skipping issue as label id 2 did not match with any of [5, 4, 3]
2024-05-02 23:29:36.763 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:199 - Skipped users: ['ajohnson1', 'ajohnson1', 'paulcheeseman', 'paulcheeseman', 'paulcheeseman', 'paulcheeseman', 'ajohnson1'] because they don't exist in the ${expected_

Issue: https://github.com/eclipse-openj9/openj9/issues/17869
Actual component: comp:vm
Actual owner: babsingh
Issue: https://github.com/eclipse-openj9/openj9/issues/11116
Actual component: comp:build
Actual owner: babsingh


2024-05-02 23:29:36.995 | INFO     | triagerx.system.triagerx:get_recommendation:70 - Recommended developers: ['babsingh', 'JasonFengJ9', 'keithc-ca', 'tajila', 'hangshao0']
2024-05-02 23:29:36.997 | DEBUG    | triagerx.system.triagerx:get_recommendation:72 - Generating recommendation by similarity...
2024-05-02 23:29:37.014 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:199 - Skipped users: ['r30shah', 'r30shah'] because they don't exist in the ${expected_developers} list
2024-05-02 23:29:37.015 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:165 - [('JasonFengJ9', 13.697406351566315), ('ChengJin01', 8.472741544246674), ('babsingh', 5.730522871017456), ('pshipton', 4.5692238211631775), ('fengxue-IS', 4.359413534402847), ('keithc-ca', 3.64415243268013), ('ThanHenderson', 3.5303089767694473)]
2024-05-02 23:29:37.017 | INFO     | triagerx.system.triagerx:get_recommendation:81 - Recommended developers by issue similarity: ['JasonFengJ9', 'ChengJ

...

2024-05-02 23:30:11.739 | INFO     | triagerx.system.triagerx:get_recommendation:70 - Recommended developers: ['gacholio', 'fengxue-IS', 'tajila', 'babsingh', 'pshipton']
2024-05-02 23:30:11.740 | DEBUG    | triagerx.system.triagerx:get_recommendation:72 - Generating recommendation by similarity...
2024-05-02 23:30:11.753 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:165 - [('gacholio', 0.6315175443887711)]
2024-05-02 23:30:11.754 | INFO     | triagerx.system.triagerx:get_recommendation:81 - Recommended developers by issue similarity: ['gacholio']
2024-05-02 23:30:11.755 | DEBUG    | triagerx.system.triagerx:get_recommendation:83 - Aggregating ranking...
2024-05-02 23:30:11.757 | INFO     | triagerx.system.triagerx:get_recommendation:90 - Recommended developers by ranking aggregation: ['gacholio', 'fengxue-IS', 'tajila']
2024-05-02 23:30:11.758 | DEBUG    | triagerx.system.triagerx:get_recommendation:58 - Processing issue...
2024-05-02 23:30:11.762 | DEBUG    

Issue: https://github.com/eclipse-openj9/openj9/issues/14495
Actual component: comp:test
Actual owner: llxia
Issue: https://github.com/eclipse-openj9/openj9/issues/8514
Actual component: comp:vm
Actual owner: babsingh


2024-05-02 23:30:11.978 | INFO     | triagerx.system.triagerx:get_recommendation:70 - Recommended developers: ['keithc-ca', 'babsingh', 'tajila', 'gacholio', 'JasonFengJ9']
2024-05-02 23:30:11.979 | DEBUG    | triagerx.system.triagerx:get_recommendation:72 - Generating recommendation by similarity...
2024-05-02 23:30:11.999 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:199 - Skipped users: ['Hello71', 'Hello71', 'Hello71', 'Hello71', 'cedrichansen', 'cedrichansen', 'Hello71', 'Hello71', 'dnakamura', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'Hello71', 'cedrichansen'] because they don't exist in the ${expected_developers} list
2024-05-02 23:30:12.000 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:165 - [('gacholio', 31.707414537668228), ('babsingh', 8.5605338960886), ('pshipton', 7.470574826002121), ('keithc-ca', 6.99490562081337)]
2024-05-02 23:30:12.002 | INFO     | triagerx.system.triagerx:get_recommenda

Issue: https://github.com/eclipse-openj9/openj9/issues/13848
Actual component: comp:vm
Actual owner: hangshao0
Issue: https://github.com/eclipse-openj9/openj9/issues/7349
Actual component: comp:vm
Actual owner: keithc-ca


2024-05-02 23:30:12.235 | INFO     | triagerx.system.triagerx:get_recommendation:70 - Recommended developers: ['JasonFengJ9', 'keithc-ca', 'babsingh', 'ChengJin01', 'hangshao0']
2024-05-02 23:30:12.236 | DEBUG    | triagerx.system.triagerx:get_recommendation:72 - Generating recommendation by similarity...
2024-05-02 23:30:12.259 | WARNING  | triagerx.system.triagerx:_get_historical_contributors:199 - Skipped users: ['manqingl', 'manqingl', 'manqingl', 'manqingl', 'AlenBadel', 'AlenBadel', 'AlenBadel'] because they don't exist in the ${expected_developers} list
2024-05-02 23:30:12.261 | DEBUG    | triagerx.system.triagerx:_get_recommendation_by_similarity:165 - [('keithc-ca', 10.225741192698479), ('hangshao0', 6.606559157371521), ('tajila', 3.2452626526355743), ('gacholio', 1.7246094346046448), ('pshipton', 1.6987474262714386), ('fengxue-IS', 1.6607587039470673)]
2024-05-02 23:30:12.262 | INFO     | triagerx.system.triagerx:get_recommendation:81 - Recommended developers by issue similar

In [24]:
dev_model_score = len(recommendations_matched) / len(test_df)
ranked_score = len(ranked_matched) / len(test_df)
component_model_score = len(component_matched) / len(test_df)

print("Top 3 Recommendation Score")
print("=============================================")
print(
    f"Developer Recommendation Model: {dev_model_score}\n"
    f"Ranking Aggregation Score: {ranked_score}\n"
    f"Component Prediction Score: {component_model_score}\n"
)

Top 3 Recommendation Score
Developer Recommendation Model: 0.6027874564459931
Ranking Aggregation Score: 0.662020905923345
Component Prediction Score: 0.9790940766550522



In [25]:
pprint(ranked_matched[29])

{'actual_component': 'comp:vm',
 'actual_owner': 'JasonFengJ9',
 'combined_ranking': ['babsingh', 'JasonFengJ9', 'pshipton'],
 'comp_prediction_score': [26.287517547607422,
                           -0.06778323650360107,
                           -5.241384983062744],
 'dev_prediction_score': [10.136852264404297,
                          3.2457101345062256,
                          2.7656984329223633,
                          1.3449610471725464,
                          0.5887564420700073],
 'issue_url': 'https://github.com/eclipse-openj9/openj9/issues/11206',
 'model_pred_status': 'Matched',
 'predicted_components': ['comp:vm', 'comp:build', 'comp:gc'],
 'predicted_developers': ['JasonFengJ9', 'pshipton', 'babsingh'],
 'rank_status': 'Matched',
 'similar_devs': [('babsingh', 16.359554082155228),
                  ('fengxue-IS', 11.67458213865757),
                  ('pshipton', 9.92135003209114),
                  ('JasonFengJ9', 9.318062141537666),
                  ('gacholio',